In [1]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd

In [2]:
df = pd.read_csv("intern_task.csv")
df.isna().sum().unique()#Проверка на пропуски. 
#Нашлось только 0, т.е пропусков нет ни в одной из фичей.

array([0], dtype=int64)

In [3]:
#В фичах 134+ видны крайне большие значения по сравнению с остальными
#Но т.к я буду использовать CatBoost могу не выполнять нормирование
#Пример с нормированием все равно оставил снизу
df.describe()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
count,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,...,2.352580e+05,2.352580e+05,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000,235258.000000
mean,0.677869,14828.413401,1.911960,0.206233,1.189847,0.550272,1.960082,0.803964,0.097557,0.541174,...,4.763432e+02,1.046698e+04,0.070299,0.740710,11.067947,0.281747,0.027033,0.000269,22.457910,9.560379
std,0.830033,8193.945170,1.237374,0.579089,1.037233,0.790947,1.203534,0.339955,0.266035,0.419973,...,1.954840e+04,2.649584e+06,0.100721,0.352800,15.336861,0.392089,0.033351,0.002088,63.708018,14.347378
min,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8215.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.666667,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.620522,0.000000,0.000000,0.006703,0.000000,4.000000,2.000000
50%,0.000000,14935.000000,2.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.500000,...,0.000000e+00,0.000000e+00,0.000000,0.926779,9.448900,0.000000,0.017761,0.000008,12.000000,5.500000
75%,1.000000,21580.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1.000000,...,1.000000e+00,4.000000e-01,0.142857,0.999613,16.146733,0.500000,0.034954,0.000109,28.000000,12.000000
max,4.000000,29995.000000,31.000000,18.000000,27.000000,9.000000,31.000000,1.000000,1.000000,1.000000,...,1.731553e+06,8.000000e+08,0.818182,1.000000,2506.799764,1.000000,0.711261,0.250000,15074.000000,1552.000000


In [4]:
train_df, test_df = train_test_split(df, random_state = 42)#Разделение на тест трейн

In [5]:
test_df = test_df.sort_values(by="query_id")#Подготовил данные для работы с CatBoost
train_df = train_df.sort_values(by="query_id")

In [6]:
X_train = train_df.drop(["rank", "query_id"], axis=1).values
y_train = train_df.iloc[:,0].values.astype("float64")
queries_train = train_df.iloc[:,1].values
#Также подготавливаю данные для работы с CatBoost
X_test = test_df.drop(["rank", "query_id"], axis=1).values
y_test = test_df.iloc[:,0].values.astype("float64")
queries_test = test_df.iloc[:,1].values

In [7]:
from collections import Counter
Counter(y_train).items()
max_relevance = np.max(y_train)
y_train /= max_relevance
y_test /= max_relevance
num_queries = np.unique(queries_train).shape[0]
#Перевёл y_train и y_test в диапазон [0, 1]

In [8]:
# from sklearn.preprocessing import StandardScaler
# Scaler = StandardScaler()
# X_train = Scaler.fit_transform(X_train)
# X_test = Scaler.fit_transform(X_test)

In [9]:
#Создаю пулы для трейна и теста, чтобы гагрузить их в CatBoost
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [10]:
#Параметры для CatBoost
default_parameters = {
    #'task_type':'GPU',
    'iterations': 5000,
    'eval_metric': 'NDCG:top=5',
    'custom_metric': ['MRR:top=5', 'PFound:top=5', 'AverageGain:top=5'],
    'verbose': 500,
    'random_seed': 0,
}


In [11]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    """
    Функция для тренировки CatBoost
    loss_function: функция потерь(в нашем случае для ранжирования)
    additional_params: доп. параметры для CatBoost(наподобие task_type, eval_metric и т.д.)
    train&test pool's: тренировочный и тестовый датасет.
    """
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [12]:
model = fit_model('QueryRMSE', default_parameters)#, {'custom_metric': 'NDCG:top=5'})

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.4032977	best: 0.4032977 (0)	total: 175ms	remaining: 14m 36s
500:	test: 0.6118630	best: 0.6122195 (493)	total: 14.6s	remaining: 2m 11s
1000:	test: 0.6190247	best: 0.6198954 (966)	total: 29s	remaining: 1m 55s
1500:	test: 0.6239002	best: 0.6241760 (1476)	total: 43.2s	remaining: 1m 40s
2000:	test: 0.6263137	best: 0.6263947 (1999)	total: 57.3s	remaining: 1m 25s
2500:	test: 0.6293013	best: 0.6298090 (2480)	total: 1m 11s	remaining: 1m 11s
3000:	test: 0.6310849	best: 0.6319251 (2961)	total: 1m 26s	remaining: 57.4s
3500:	test: 0.6322284	best: 0.6328210 (3458)	total: 1m 40s	remaining: 43.1s
4000:	test: 0.6343195	best: 0.6346977 (3951)	total: 1m 55s	remaining: 28.8s
4500:	test: 0.6354660	best: 0.6359503 (4357)	total: 2m 10s	remaining: 14.4s
4999:	test: 0.6355041	best: 0.6362405 (4696)	total: 2m 24s	remaining: 0us

bestTest = 0.636240538
bestIteration = 4696

Shrink model to first 4697 iterations.


In [13]:
for key, value in model.get_best_score()["validation"].items():
    print(f"Результат модели для {key}: {value}")

Результат модели для PFound:top=5: 0.6151296876623105
Результат модели для NDCG:top=5;type=Base: 0.636240537957036
Результат модели для QueryRMSE: 0.17311922573974273
Результат модели для AverageGain:top=5: 0.2657112393376828
Результат модели для MRR:top=5: 0.20503428667001136
